# BM-25 and Query Similarity Ranking

This notebook uses code for BM-25 notebook https://www.kaggle.com/dgunning/browsing-research-papers-with-a-bm25-search-engine.
It uses relevant keywords from https://www.kaggle.com/d4v1d3/cord-19-lda-topic-modeling-reccomendation-system and performs a BM-25 search on the keywords.

Then, the results are weighted against ranking of sentences from https://colab.research.google.com/drive/1DzjAfXIC8wQg6eqVq3QoMX12OuG4VtGW. This encourages the chosen sentences to be from documents most relevant to COVID-19.

### Import Libraries

In [1]:
!pip install rank_bm25 nltk

  Created wheel for rank-bm25: filename=rank_bm25-0.2-py3-none-any.whl size=4161 sha256=29ca5298e04df3dc1062815525329fadb3b56cc38bce63f9dd59154a4eef1072
  Stored in directory: /root/.cache/pip/wheels/82/a9/a2/33d3c389435f63f6fa490ef91417c997ccc4e7db75f5a7b2f8
Successfully built rank-bm25


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path, PurePath
import requests
from requests.exceptions import HTTPError, ConnectionError
from ipywidgets import interact
import ipywidgets as widgets
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
nltk.download("punkt")
import re
import os
import json
import glob
import sys
from tqdm import tqdm
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Get Data

In [3]:
sys.path.insert(0, "../")

DATA_PATH = '/kaggle/input/CORD-19-research-challenge'

# Walks all subdirectories in a directory, and their files. 
# Opens all json files we deem relevant, and append them to
# a list that can be used as the "data" argument in a call to 
# pd.DataFrame.
def gather_jsons(dirName):
    
    # Get the list of all files in directory tree at given path
    # include only json with encoded id (40-character SHA hash)
    # Only length of filename is checked, but this should be sufficient
    # given the task.
    
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames
                        if file.endswith("json")
                        and len(file) == 45]
    jsons = []
    
    print(str(len(listOfFiles)) + " jsons found! Attempting to gather.")
    
    for file in tqdm(listOfFiles):
        with open(file) as json_file:
            jsons.append(json.load(json_file))
    return jsons
        
        
# Returns a dictionary object that's easy to parse in pandas.
def extract_from_json(json):
    
    # For text mining purposes, we're only interested in 4 columns:
    # abstract, paper_id (for ease of indexing), title, and body text.
    # In this particular dataset, some abstracts have multiple sections,
    # with ["abstract"][1] or later representing keywords or extra info. 
    # We only want to keep [0]["text"] in these cases. 
    if len(json["abstract"]) > 0:
        json_dict = {
            "_id": json["paper_id"],
            "title": json["metadata"]["title"],
            "abstract": json["abstract"],
            "text": " ".join([i["text"] for i in json["body_text"]])
        }
        
    # Else, ["abstract"] isn't a list and we can just grab the full text.
    else:
        json_dict = {
            "_id": json["paper_id"],
            "title": json["metadata"]["title"],
            "abstract": json["abstract"],
            "text": " ".join([i["text"] for i in json["body_text"]])
        }

    return json_dict

# Combines gather_jsons and extract_from_json to create a
# pandas DataFrame object.
def gather_data(dirName):
    return(pd.DataFrame(data=[extract_from_json(json) for json in gather_jsons(dirName)]))

corona_df = gather_data(f"{DATA_PATH}")
corona_df['abstract'] = corona_df.abstract.apply(lambda x: ' '.join([r['text'] for r in x]))
corona_df.to_csv("covid_data_full.csv", index=False)

  0%|          | 26/29315 [00:00<01:56, 251.63it/s]

29315 jsons found! Attempting to gather.


100%|██████████| 29315/29315 [02:36<00:00, 187.25it/s]


### Text Preprocessing
To prepare the text for the search index we perform the following steps
1. Remove punctuations and special characters
2. Convert to lowercase
3. Tokenize into individual tokens (words mostly)
4. Remove stopwords like (and, to))

You can tweak the code below to improve the search results

In [4]:
english_stopwords = list(set(stopwords.words('english')))

def strip_characters(text):
    t = re.sub('\(|\)|:|,|;|\.|’|”|“|\?|%|>|<', '', text)
    t = re.sub('/', ' ', t)
    t = t.replace("'",'')
    return t

def clean(text):
    t = text.lower()
    t = strip_characters(t)
    return t

def tokenize(text):
    words = nltk.word_tokenize(text)
    return list(set([word for word in words 
                     if len(word) > 1
                     and not word in english_stopwords
                     and not (word.isnumeric() and len(word) is not 4)
                     and (not word.isnumeric() or word.isalpha())] )
               )

def preprocess(text):
    t = clean(text)
    tokens = tokenize(t)
    return tokens

class SearchResults:
    
    def __init__(self, 
                 data: pd.DataFrame,
                 columns = None):
        self.results = data
        if columns:
            self.results = self.results[columns]
            
    def __getitem__(self, item):
        return Paper(self.results.loc[item])
    
    def __len__(self):
        return len(self.results)
        
    def _repr_html_(self):
        return self.results._repr_html_()

SEARCH_DISPLAY_COLUMNS = ['_id', 'title', 'abstract', 'text']

class WordTokenIndex:
    
    def __init__(self, 
                 corpus: pd.DataFrame, 
                 columns=SEARCH_DISPLAY_COLUMNS):
        self.corpus = corpus
        raw_search_str = self.corpus.abstract.fillna('') + ' ' + self.corpus.title.fillna('') + ' ' + self.corpus.text.fillna('')
        self.index = raw_search_str.apply(preprocess).to_frame()
        self.index.columns = ['terms']
        self.index.index = self.corpus.index
        self.columns = columns
    
    def search(self, search_string):
        search_terms = preprocess(search_string)
        result_index = self.index.terms.apply(lambda terms: any(i in terms for i in search_terms))
        results = self.corpus[result_index].copy().reset_index().rename(columns={'index':'paper'})
        return SearchResults(results, self.columns + ['paper'])

## Using a RankBM25 Search Index

RankBM25 is a python library that implements algorithms for a simple search index.
https://pypi.org/project/rank-bm25/

In [5]:
class RankBM25Index(WordTokenIndex):
    
    def __init__(self, corpus: pd.DataFrame, columns=SEARCH_DISPLAY_COLUMNS):
        super().__init__(corpus, columns)
        self.bm25 = BM25Okapi(self.index.terms.tolist())
        
    def search(self, search_string, n=4):
        search_terms = preprocess(search_string)
        doc_scores = self.bm25.get_scores(search_terms)
        ind = np.argsort(doc_scores)[::-1][:n]
        results = self.corpus.iloc[ind][self.columns]
        results['Score'] = doc_scores[ind]
        results['orig_ind'] = ind
        results = results[results.Score > 0]
        return SearchResults(results.reset_index(), self.columns + ['Score', 'orig_ind'])


## The Search Index

Creating a search index on the entire dataset takes over a minute. As a todo I will try to speed it up.
For now let's create the search index on the first 10000 records

In [6]:
bm25_index = RankBM25Index(corona_df)

These keywords were retrieved from an LDA kernel and seem most relevant to COVID-19.

In [7]:
keywords = ['sars-cov', 'sars', 'coronavirus', 'ace2', 'coronaviruses', 'ncov', 'covid-19', 'wuhan', 'spike', 'sars-cov-2']

The chosen rows are the union of 100 BM25 results for each keyword. We keep the scores for ranking later.

In [8]:
results = None
added = []
for word in keywords:
    word_result = bm25_index.search(word, n=100).results
    if results is None:
        results = word_result
        added += [r.orig_ind for i, r in word_result.iterrows()]
        continue
    for i, r in word_result.iterrows():
        if r.orig_ind not in added:
            results = results.append(r)
            added.append(r.orig_ind)
df = results.sort_values(by='Score', ascending=False)
df.reset_index(drop=True, inplace=True)

# Ranking

In [9]:
# !tar -xvf /kaggle/input/spacy-covid19/covid-19-en_lg.tar.xz

In [10]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import scispacy
import spacy
import en_core_sci_lg
nlp = en_core_sci_lg.load()
# nlp = spacy.load("covid-19-en_lg/")
nlp.max_length=2000000

     |████████████████████████████████| 3.0 MB 11.1 MB/s 
     |████████████████████████████████| 12.9 MB 20.7 MB/s 
     |████████████████████████████████| 6.0 MB 52.2 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
  Created wheel for scispacy: filename=scispacy-0.2.4-py3-none-any.whl size=35203 sha256=4b6568239bf845728a5ad9d2df15a8eda32bf48bf2658b5bec1da31c44f54fe0
  Stored in directory: /root/.cache/pip/wheels/80/01/69/37a2ab4f9b61773c187d83257ffb31365a5ad57e7779ae5e92
Successfully built scispacy
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.15.13
    Uninstalling botocore-1.15.13:
      Successfully uninstalled botocore-1.15.13
  Attempting uninstall: rsa
    Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
     |████████████████████████████████| 500.6 MB 5.1 kB/s 
  Created whe

In [11]:
from tqdm import tqdm

vector_list = []
for i in tqdm(df.index):
    doc = nlp(df.iloc[i].text)
    sents = [sent for sent in doc.sents]
    vecs = [sent.vector for sent in sents]
    for j in range(len(sents)):
        vector_list.append(
            {"_id": df.iloc[i]._id, 
             "score": df.iloc[i]['Score'],
             "sentence": j, 
             "vector": vecs[j], 
             "start_span": sents[j].start_char,
             "end_span": sents[j].end_char})
vector_df = pd.DataFrame(data=vector_list)

100%|██████████| 609/609 [02:11<00:00,  4.65it/s]


In [12]:
queries = """Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.
Prevalence of asymptomatic shedding and transmission. 
Prevalence of asymptomatic shedding and transmission in children, infants, and young people.
Seasonality of transmission of the virus.
Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic or hydrophobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).
Persistence and stability on a multitude of substrates and sources (nasal discharge, sputum, urine, fecal matter, blood, bodily fluids and secretions).
Persistence of virus on surfaces of different materials (copper, stainless steel, plastic).
Natural history of the virus and shedding of it from an infected person.
Implementation of diagnostics and products to improve clinical processes.
Disease models, including animal models for infection, disease and transmission.
Tools and studies to monitor phenotypic change and potential adaptation of the virus.
Immune response and immunity to the virus.
Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings.
Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings.
Role of the environment in transmission."""
queries = queries.splitlines()
queries_df = pd.DataFrame(data=[{"query":query} for query in queries])

In [13]:
query_vector_list = []
for i in tqdm(range(len(queries))):
    doc = nlp(queries[i])
    vec = doc.vector
    query_vector_list.append({"_id": f"query_{i}", "vector": vec})
    
query_vector_df = pd.DataFrame(data=query_vector_list)
query_vector_df.to_csv("query_vecs.csv",index=False)

100%|██████████| 15/15 [00:00<00:00, 75.30it/s]


Here, we use a 0.9 RATIO to give weight to the relevance score. The final score is a linear combination of the cosine similarity score and the relevance score from BM25 search.

In [14]:
from scipy.spatial import distance
RATIO = 0.9
distances = distance.cdist([value for value in query_vector_df["vector"]], [value for value in vector_df["vector"].values], "cosine")
w2v_searchable_df = vector_df.drop(columns=["vector"])
# Create a column with cosine distances for each query vs the sentence
for i in range(len(queries)):
    #w2v_searchable_df[f"query_{i}_distance"] = 1 - (np.power((1 - distances[i]), RATIO) * w2v_searchable_df['score'])
    w2v_searchable_df[f"query_{i}_distance"] = RATIO * (1 - distances[i]) + (1-RATIO) * w2v_searchable_df['score']
w2v_searchable_df.to_csv("covid_w2v_searchable.csv", index=False)

In [15]:
for i in range(len(queries)):
    columnName = f"query_{i}_distance"
    context = w2v_searchable_df.sort_values(by=columnName, ascending=False)[["_id","start_span","end_span"]][:20]
    ix = context["_id"].to_list()
    spans1 = context["start_span"].to_list()
    spans2 = context["end_span"].to_list()
    print("Question: " + queries[i] + "\n")
    for j in range(len(context.index)):
        score = df[df["_id"] == ix[j]].iloc[0]['Score']
        print(f"Rank {j+1} (Relevance Score {score}): " + str(df[df["_id"] == ix[j]].iloc[0]["text"])[spans1[j]:spans2[j]] + "\n")

Question: Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.

Rank 1 (Relevance Score 8.870345893837603): [3] , which indicated the likelihood of travel related risks of 2019-nCoV spreading, we suspected the spread of infections could be associated with the domestic transportations in mainland China.

Rank 2 (Relevance Score 8.935709343048465): At the beginning of the outbreak, incidence of NCP may be vastly underestimated because of asymptomatic infection, no timely diagnosis, inadequate supplies and low sensitiviy of nCov Kit.

Rank 3 (Relevance Score 8.935709343048465): Chinese health authorities did an immediate investigation to characterize and control the disease, including isolation of suspected cases, monitoring of close contacts, collection of epidemiological and clinical data from confirmed patients, and development of diagnostic and treatment procedures [2] .

